### Ejercicio 26 Julio
---
- Realizar el analisis exploratorio de data (duplicados, nulos, columnas erroneas). 
- Luego realizar 4 gráficos con sus correspondientes explicaciones, tengan en cuenta la estetica de las ilustraciones. 
- Bajo su concepto, generar una salida OUTCOME de clasificación, a criterio de cada uno, y una columna PREDICTION, aleatoria con lo que seria el resultado de un modelo.
- Generar la matriz de confusión, calcular el RECALL. 
- Generar una presentación con 4 slides, titulo, objetivos, metodologia, graficos y 2 conclusiones.

In [40]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

#### 1.EDA (Analisis Eploratorio)

In [41]:
file = r'C:/Users/Juan.Paredes/Unilever/Data & Analytics CL DEV - General/Nacho/1. Visual Studio Scripts/CURSO DOJO/1.-Curso-Coding-Dojo/1. Ejercicio 26 Julio/nutrients_csvfile.csv'
df = pd.read_csv(file, sep=",",thousands=",",decimal=".")
df.head()

,Food,Measure,Grams,Calories,Protein,Fat,Sat.Fat,Fiber,Carbs,Category
0,Cows' milk,1 qt.,976,660,32,40,36,0,48,Dairy products
1,Milk skim,1 qt.,984,360,36,t,t,0,52,Dairy products
2,Buttermilk,1 cup,246,127,9,5,4,0,13,Dairy products
3,"Evaporated, undiluted",1 cup,252,345,16,20,18,0,24,Dairy products
4,Fortified milk,6 cups,1419,"1,373",89,42,23,1.4,119,Dairy products


In [42]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 335 entries, 0 to 334
Data columns (total 10 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Food      335 non-null    object
 1   Measure   335 non-null    object
 2   Grams     335 non-null    int64 
 3   Calories  334 non-null    object
 4   Protein   335 non-null    object
 5   Fat       335 non-null    object
 6   Sat.Fat   333 non-null    object
 7   Fiber     335 non-null    object
 8   Carbs     335 non-null    object
 9   Category  335 non-null    object
dtypes: int64(1), object(9)
memory usage: 26.3+ KB


##### Resumen EDA
---
1. Tenemos 10 columnas y 335 registros.
2. Vemos que las columnas de Calories / Protein / Fat / Sat.Fat / Carbs no son numericas, sino string.
3. Pese a que pusimos separador de columnas y decimales al leer el archivo, estas columnas no quedaron en formato numerico, sino que como string, esto quiere decir que hay errores en la data de estas columnas.
4. Estos errores pueden ser de valores mal digitados o valores que no son numericos sino letras o guiones.
5. Haremos una revision por cada una de estas columnas.

In [43]:
df[['Food','Fiber']][df['Fiber'].str.isnumeric() == False] #vemos como en la columna Fiberm tenemos valores con una "t", que al parecer no corresponden a esta columna.

,Food,Fiber
4,Fortified milk,1.4
51,Corned beef hash canned,t
53,Pot-pie,t
54,Corned beef hash Stew,t
81,Mackerel,a
...,...,...
317,"Peanut butter, natural",0.9
318,Peanuts,1.2
319,Pecans,1.1
320,Sesame seeds,3.1


In [44]:
# Hacemos una revision de cada una de estas columnas, tratando de convertirlas a formato numerico, contando los valores a los cuales no se les pudo aplicar la conversion:

columnas = ['Protein','Fat','Sat.Fat','Fiber','Carbs']

for i in columnas:
    x = df[i][df[i].str.isdecimal() == False].count()
    y = len(df[i])
    z = ((x/y)*100).round()
    print('La columna ' + i + ' tiene un ' + str(z) + '% de valores erroneos.')

La columna Protein tiene un 12.0% de valores erroneos.
La columna Fat tiene un 35.0% de valores erroneos.
La columna Sat.Fat tiene un 4.0% de valores erroneos.
La columna Fiber tiene un 51.0% de valores erroneos.
La columna Carbs tiene un 4.0% de valores erroneos.


In [45]:
# Realizamos la conversion a la "fuerza", osea se convertiran los valores que se pueda y el resto se dejara como NaN.
for e in columnas:
    df[e] = df[e].apply(pd.to_numeric, errors='coerce')

In [46]:
# Despues de haber aplicado la transformacion de arriba, revisamos cuantos valores quedaron NaN en cada columna.
for i in columnas:
    x = df[i].isna().sum()
    y = len(df[i])
    z = ((x/y)*100).round()
    print('La columna ' + i + ' tiene un ' + str(z) + '% de valores nulos por correccion que no pudieron ser transformados a decimales.')

La columna Protein tiene un 12.0% de valores nulos por correccion que no pudieron ser transformados a decimales.
La columna Fat tiene un 35.0% de valores nulos por correccion que no pudieron ser transformados a decimales.
La columna Sat.Fat tiene un 5.0% de valores nulos por correccion que no pudieron ser transformados a decimales.
La columna Fiber tiene un 8.0% de valores nulos por correccion que no pudieron ser transformados a decimales.
La columna Carbs tiene un 4.0% de valores nulos por correccion que no pudieron ser transformados a decimales.


In [47]:
# Vamos que la columna Fat, fue la que tuvo mayor perdida por la conversion, ya que es la que tenia mayor cantidad de valores que no se podian convertir a numero. (35% de NaN)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 335 entries, 0 to 334
Data columns (total 10 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Food      335 non-null    object 
 1   Measure   335 non-null    object 
 2   Grams     335 non-null    int64  
 3   Calories  334 non-null    object 
 4   Protein   296 non-null    float64
 5   Fat       219 non-null    float64
 6   Sat.Fat   318 non-null    float64
 7   Fiber     308 non-null    float64
 8   Carbs     322 non-null    float64
 9   Category  335 non-null    object 
dtypes: float64(5), int64(1), object(4)
memory usage: 26.3+ KB


#### 1. Analisis Data

In [53]:
corr = df.corr()
corr.style.background_gradient(cmap='coolwarm').set_precision(2)

C:\Users\Juan.Paredes\AppData\Local\Temp\ipykernel_9908\2001914525.py:2: FutureWarning: this method is deprecated in favour of `Styler.format(precision=..)`
  corr.style.background_gradient(cmap='coolwarm').set_precision(2)


,Grams,Protein,Fat,Sat.Fat,Fiber,Carbs
Grams,1.00,0.21,0.06,0.05,0.06,0.35
Protein,0.21,1.00,0.76,0.79,0.84,0.41
Fat,0.06,0.76,1.00,0.98,0.75,0.34
Sat.Fat,0.05,0.79,0.98,1.00,0.80,0.29
Fiber,0.06,0.84,0.75,0.80,1.00,0.46
Carbs,0.35,0.41,0.34,0.29,0.46,1.00


In [54]:
# Fill diagonal and upper half with NaNs
mask = np.zeros_like(corr, dtype=bool)
mask[np.triu_indices_from(mask)] = True
corr[mask] = np.nan
(corr
 .style
 .background_gradient(cmap='Greens', axis=None, vmin=-1, vmax=1)
 .highlight_null(null_color='#f1f1f1')  # Color NaNs grey
 .set_precision(2))

C:\Users\Juan.Paredes\AppData\Local\Temp\ipykernel_9908\4137461140.py:5: FutureWarning: this method is deprecated in favour of `Styler.format(precision=..)`
  (corr


,Grams,Protein,Fat,Sat.Fat,Fiber,Carbs
Grams,nan,nan,nan,nan,nan,nan
Protein,0.21,nan,nan,nan,nan,nan
Fat,0.06,0.76,nan,nan,nan,nan
Sat.Fat,0.05,0.79,0.98,nan,nan,nan
Fiber,0.06,0.84,0.75,0.80,nan,nan
Carbs,0.35,0.41,0.34,0.29,0.46,nan


In [ ]:
df.groupby('Category')['Category'].count().sort_values()

Category
Fruits R-Z                           8
Jams, Jellies                        8
Soups                               10
Drinks,Alcohol, Beverages           12
Seeds and Nuts                      12
Fats, Oils, Shortenings             14
Vegetables F-P                      14
Fish, Seafood                       19
Fruits A-F                          22
Dairy products                      28
Fruits G-P                          28
Vegetables A-E                      28
Vegetables R-Z                      28
Desserts, sweets                    29
Meat, Poultry                       30
Breads, cereals, fastfood,grains    45
Name: Category, dtype: int64

#### 2. Generar 4 Graficos